In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
#1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv ("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,64.44,55,90,10.36,overcast clouds
1,1,Saint George,US,37.1041,-113.5841,73.99,78,1,4.61,clear sky
2,2,Port Elizabeth,ZA,-33.9180,25.5701,71.10,26,0,8.05,clear sky
3,3,Evensk,RU,61.9500,159.2333,59.52,76,87,4.47,overcast clouds
4,4,Kapaa,US,22.0752,-159.3190,88.12,74,1,4.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,64.44,55,90,10.36,overcast clouds
3,3,Evensk,RU,61.9500,159.2333,59.52,76,87,4.47,overcast clouds
5,5,Kaitangata,NZ,-46.2817,169.8464,53.92,49,6,9.19,clear sky
6,6,Ilulissat,GL,69.2167,-51.1000,53.62,58,40,6.91,drizzle
9,9,Kiama,AU,-34.6833,150.8667,67.57,52,5,8.99,clear sky
10,10,Mataura,NZ,-46.1927,168.8643,51.48,78,22,4.47,few clouds
14,14,Yellowknife,CA,62.4560,-114.3525,68.52,52,40,7.00,scattered clouds
15,15,Kodiak,US,57.7900,-152.4072,62.11,63,90,5.75,overcast clouds
20,20,Cape Town,ZA,-33.9258,18.4232,58.12,64,0,1.99,clear sky
22,22,Khatanga,RU,71.9667,102.5000,67.35,57,100,18.77,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
551,False,False,False,False,False,False,False,False,False,False
557,False,False,False,False,False,False,False,False,False,False
558,False,False,False,False,False,False,False,False,False,False
559,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
preferred_cities_df.dropna(inplace = True) 
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,64.44,55,90,10.36,overcast clouds
1,1,Saint George,US,37.1041,-113.5841,73.99,78,1,4.61,clear sky
2,2,Port Elizabeth,ZA,-33.9180,25.5701,71.10,26,0,8.05,clear sky
3,3,Evensk,RU,61.9500,159.2333,59.52,76,87,4.47,overcast clouds
4,4,Kapaa,US,22.0752,-159.3190,88.12,74,1,4.00,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Norman Wells,CA,64.44,overcast clouds,65.2820,-126.8329,
1,Saint George,US,73.99,clear sky,37.1041,-113.5841,
2,Port Elizabeth,ZA,71.10,clear sky,-33.9180,25.5701,
3,Evensk,RU,59.52,overcast clouds,61.9500,159.2333,
4,Kapaa,US,88.12,clear sky,22.0752,-159.3190,
5,Kaitangata,NZ,53.92,clear sky,-46.2817,169.8464,
6,Ilulissat,GL,53.62,drizzle,69.2167,-51.1000,
7,Edson,CA,77.14,clear sky,53.5835,-116.4356,
8,Handan,CN,85.62,light rain,36.6006,114.4678,
9,Kiama,AU,67.57,clear sky,-34.6833,150.8667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}" 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")  
 

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Norman Wells,CA,64.44,overcast clouds,65.2820,-126.8329,Heritage Hotel
1,Saint George,US,73.99,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
2,Port Elizabeth,ZA,71.10,clear sky,-33.9180,25.5701,39 On Nile Guest House
3,Evensk,RU,59.52,overcast clouds,61.9500,159.2333,
4,Kapaa,US,88.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
558,Svetlyy,RU,66.25,scattered clouds,54.6750,20.1347,"Gostinitsa-Kafe, Alena"
559,Phek,IN,63.23,overcast clouds,25.6667,94.5000,Prayer Centre
560,Yelan,RU,60.31,clear sky,50.9490,43.7378,Gostinitsa Na Sovetskoy
561,Balikpapan,ID,79.79,few clouds,-1.2675,116.8289,ibis Balikpapan


In [12]:
# 8a. Create the output File (CSV)
clean_hotel_df.to_csv("Weatherpy_vacation.csv")
# 8b. Export the City_Data into a csv
output_data_file = "Weatherpy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))